In [29]:
from io import BytesIO, StringIO
from pathlib import Path
from zipfile import ZipFile

import pandas as pd
from PIL import Image

def list_of_videos(file_name, data_zip_dir = './data'):
    
    _zip_file_path = Path(data_zip_dir) / file_name # zamień na ścieżkę 
    _list_of_videos = [i[17:-4] for i in ZipFile(_zip_file_path).namelist()
                                    if i.startswith('data/annotations/') and i.endswith('csv')]
    return _list_of_videos
    
def get_video(video_name: str, file_name, data_zip_dir = './data') -> pd.DataFrame:
    
    """Function gets name of video file and returns dataframe with coordinates for hands and balls localization"""
    
    # assert allows to check if video_name is in our files
    assert video_name in list_of_videos(file_name, data_zip_dir), f"Video named '{video_name}' wasn't found in zip file '{_zip_file_path.name}'"
    
    _zip_file_path = Path(data_zip_dir) / file_name
    zip = ZipFile(_zip_file_path)
    video_data = pd.read_csv(
       StringIO(str(zip.open(f"{'data/annotations/'}{video_name}.csv").read(), 'utf-8')),
       header=None,
       names='file_name right_hand_x right_hand_y left_hand_x left_hand_y ball_1_x'
             ' ball_1_y ball_2_x ball_2_y ball_3_x ball_3_y'.split()
    )
    video_data['photo'] = None
    for i, photo_name in video_data['file_name'].iteritems():
       video_data.loc[i, 'photo'] = Image.open(BytesIO(zip.open('data/frames/' + photo_name).read()))
    return video_data


In [30]:
"""Assertion tests usefull in refactoring"""

result_list_of_videos = ['2-dark-mix-outside-wall','1-green-circus-blue-mat','3-dark-mix-stenkrossen-silverscreen','1-blue-red-circus-blue-chair','3-blue-outside-door1','3-j10-white-panels','2-j9-changing-room','3-blue-circus-big-doors','2-j12-armchair','3-mixed-j1-basement-shelf']
assert list_of_videos("ballsandhands.zip")[:10] == result_list_of_videos

result_get_video = ['file_name','right_hand_x','right_hand_y','left_hand_x','left_hand_y','ball_1_x','ball_1_y','ball_2_x','ball_2_y','ball_3_x','ball_3_y','photo']
assert list(get_video('1-apple-red-room-door', 'ballsandhands.zip')) == result_get_video
